In [7]:
import numpy as np
import pandas as pd
import scipy.stats as stats

In [2]:
data = pd.read_csv('homepage-experiment-data.csv')
data.head()

,Day,Control Cookies,Control Downloads,Control Licenses,Experiment Cookies,Experiment Downloads,Experiment Licenses
0,1,1764,246,1,1850,339,3
1,2,1541,234,2,1590,281,2
2,3,1457,240,1,1515,274,1
3,4,1587,224,1,1541,284,2
4,5,1606,253,2,1643,292,3


## Check Invariant Metric

Check the number of cookies assigned to each group

$$H_0: \mu_{cont} = 0.5$$
$$H_0: \mu_{cont} \neq 0.5$$

In [22]:
n_control = data['Control Cookies'].sum()
n_control

46851

In [23]:
n_obs = data['Experiment Cookies'].sum() + n_control
n_obs

94197

In [24]:
# should have 50% prob of being assigned to control homepage
p = 0.5

sd = np.sqrt(p * (1-p) * n_obs)

# with continuity correction
z = ((n_control + 0.5) - p * n_obs) / sd

print(z)
print(2 * stats.norm.cdf(z))

-1.6095646049678511
0.10749294050130412


In [36]:
# simulation approach

# # simulate outcomes under null, compare to observed outcome
p = 0.5
n_trials = 200_000

samples = np.random.binomial(n_obs, p, n_trials)

p_val = np.logical_or(samples <= n_control, samples >= (n_obs - n_control)).mean()

print(p_val)

0.10786


In [34]:
# alternate simulation approach (for 2 sided hypothesis)
print(n_control)

null_mean = 0.5 * n_obs

print(null_mean)

46851
47098.5


In [35]:
null_mean = 0.5 * n_obs

# since n_control is lower than null_mean
lower_bound = n_control
upper_bound = null_mean + (null_mean - n_control)

prob_more_extreme_high = (samples > upper_bound).mean()
prob_more_extreme_low = (samples < lower_bound).mean()

pval = prob_more_extreme_low + prob_more_extreme_high
pval

0.104965

## Check Evaluation Metric

Download Rate:
$$H_0: \mu_{exper} - \mu_{cont} \leq 0$$
$$H_0: \mu_{exper} - \mu_{cont} > 0$$

License Purchasing Rate:
$$H_0: \mu_{exper} - \mu_{cont} \leq 0$$
$$H_0: \mu_{exper} - \mu_{cont} > 0$$

In [19]:
# Check evaluation metric: download rate
n_downloads_control = data['Control Downloads'].sum()
n_downloads_experiment = data['Experiment Downloads'].sum()

n_cookies_control = data['Control Cookies'].sum()
n_cookies_experiment = data['Experiment Cookies'].sum()

p_null = (n_downloads_control + n_downloads_experiment) / (n_cookies_control + n_cookies_experiment)
p_downloads_control = n_downloads_control / n_cookies_control
p_downloads_experiment = n_downloads_experiment / n_cookies_experiment

print(p_null)
print(p_downloads_control)
print(p_downloads_experiment)

0.1709396265273841
0.16123455209067042
0.180543234908968


In [20]:
# compute standard error, z-score, and p-value
se_p = np.sqrt(p_null * (1-p_null) * (1/n_cookies_control + 1/n_cookies_experiment))

z = (p_downloads_experiment - p_downloads_control) / se_p
p_val = 1-stats.norm.cdf(z)
print(z)
print(p_val)

7.870833726066236
1.7763568394002505e-15


In [21]:
# simulation approach

# simulate outcomes under null, compare to observed outcome
n_trials = 200_000

num_downloads_control = np.random.binomial(n_cookies_control, p_null, n_trials)
num_downloads_experiment = np.random.binomial(n_cookies_experiment, p_null, n_trials)
samples = num_downloads_experiment / n_cookies_experiment - num_downloads_control / n_cookies_control

p_val = (samples >= (p_downloads_experiment - p_downloads_control)).mean()
print(p_val)

0.0


In [14]:
# Check evaluation metric: license purchasing rate

n_licenses_control = data['Control Licenses'].sum()
n_licenses_experiment = data['Experiment Licenses'].sum()

# get number of cookies for each group till day 21
n_cookies_control = data[data['Day'] <= 21]['Control Cookies'].sum()
n_cookies_experiment = data[data['Day'] <= 21]['Experiment Cookies'].sum()

In [15]:
p_null = (n_licenses_control + n_licenses_experiment) / (n_cookies_control + n_cookies_experiment)
p_licenses_control = n_licenses_control / n_cookies_control
p_licenses_experiment = n_licenses_experiment / n_cookies_experiment

print(p_null)
print(p_licenses_control)
print(p_licenses_experiment)

0.021175986842105265
0.021032051661828307
0.021317490826489604


In [16]:
# compute standard error, z-score, and p-value
se_p = np.sqrt(p_null * (1-p_null) * (1/n_cookies_control + 1/n_cookies_experiment))

z = (p_licenses_experiment - p_licenses_control) / se_p
p_val = 1-stats.norm.cdf(z)
print(z)
print(p_val)

0.2586750111658684
0.3979430008399871


In [18]:
# simulation approach

# simulate outcomes under null, compare to observed outcome
n_trials = 200_000

num_licenses_control = np.random.binomial(n_cookies_control, p_null, n_trials)
num_licenses_experiment = np.random.binomial(n_cookies_experiment, p_null, n_trials)
samples = num_licenses_experiment / n_cookies_experiment - num_licenses_control / n_cookies_control

p_val = (samples >= (p_licenses_experiment - p_licenses_control)).mean()
print(p_val)

0.39982
